In [1]:
import os
import cv2
import PIL
import numpy as np
from glob import glob
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential, load_model, model_from_json, Model

In [2]:
img1 =image.load_img('/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/GrapevineLeafDataset/train/Ak/Ak (67).png')

In [3]:
train_dir ='/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/GrapevineLeafDataset/train'

In [4]:
cv2.imread('/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/GrapevineLeafDataset/train/Ak/Ak (67).png').shape

(511, 511, 3)

In [5]:
num_classes = 5

In [11]:
from keras.models import Model
from keras.layers import Flatten, Dense, BatchNormalization
from keras.applications import VGG16
#from keras.preprocessing import image

IMAGE_SIZE = [511, 511]  # we will keep the image size as (64,64). You can increase the size for better results. 

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
x = BatchNormalization()(x)
x = Dense(128, activation = 'relu')(x)
x = BatchNormalization()(x)                        # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
valid_dir ='/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/GrapevineLeafDataset/valid'

In [13]:
IMAGE_SIZE = [511, 511]

train_gen = ImageDataGenerator(
    rescale=1./255
    )
val_gen = ImageDataGenerator(
    rescale=1./255
    )  


train_set = train_gen.flow_from_directory(
    train_dir,
    class_mode = 'categorical',
    target_size = IMAGE_SIZE,
    batch_size = 16
    )
val_set = val_gen.flow_from_directory(
    valid_dir,
    class_mode = 'categorical',
    target_size = IMAGE_SIZE,
    batch_size = 16
    )

Found 5379 images belonging to 5 classes.
Found 568 images belonging to 5 classes.


In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 511, 511, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 511, 511, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 511, 511, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 255, 255, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 255, 255, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 255, 255, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 127, 127, 128)     0   

In [ ]:
VGG_model = model.fit(
     train_set,
     validation_data=val_set, 
     epochs=1,
     steps_per_epoch = len(train_set),
     validation_steps = len(val_set),)

 12/337 [>.............................] - ETA: 3:07:51 - loss: 3.4768 - accuracy: 0.2344

In [ ]:
model.save('VGG16_grapeleavestypes.h5')

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_dir = '/Users/aayushrangra/Desktop/GrapeVine_Disease_Detection/GrapevineLeafDataset/test'

In [ ]:
test_set = test_gen.flow_from_directory(
    test_dir,
    class_mode = 'categorical',
    target_size = IMAGE_SIZE,
    batch_size = 16
)

In [ ]:
inception_model.history.keys()

In [ ]:
acc = inception_model.history['accuracy']
val_acc = inception_model.history['val_accuracy']

loss = inception_model.history['loss']
val_loss = inception_model.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(1), acc, label='Training Accuracy')
plt.plot(range(1), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()


plt.subplot(1, 2, 2)
plt.plot(range(1), loss, label='Training Loss')
plt.plot(range(1), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predictions = model.predict(test_set)

In [ ]:
y_pred = np.argmax(predictions, axis=-1)

In [ ]:
test_set.class_indices

In [ ]:
scores = model.evaluate(test_set, batch_size=16, verbose=2)

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

# Assuming acc_train, acc_test, and acc_val are lists with accuracy values
# Replace these with your actual values

acc_train = [0.9576]  # Replace with your actual value for training accuracy
acc_test = [0.9759]   # Replace with your actual value for test accuracy
acc_val = [0.9816]    # Replace with your actual value for validation accuracy

# Set up the figure
fig, ax = plt.subplots()

# Define colors for each bar
colors = ['blue', 'orange', 'green']

# Bar graph for accuracy
bar_width = 0.3
bar_positions = np.arange(3)

ax.bar(bar_positions, [acc_train[0], acc_test[0], acc_val[0]], width=bar_width, alpha=0.5, color=colors, label='Accuracy')

# Set labels and ticks
ax.set_xticks(bar_positions)
ax.set_xticklabels(['Training', 'Test', 'Validation'])


plt.title('Training, Test, and Validation Accuracy')
plt.show()


